In [1]:
import json
import numpy as np
from collections import Counter

In [2]:
#读取数据
path='E://relation//train_data.json//train_data.json'
file = open(path, 'r', encoding='utf-8')
papers =[]
for line in file.readlines():
    dic = json.loads(line)
    papers.append(dic)
print(len(papers))

173108


In [5]:
Tags=['O']
for pos in ['B','M','E','S']:
    for entity in entity_type:
        Tags.append(pos+'_'+entity)
tag2id={v:k for k,v in enumerate(Tags)}
id2tag={k:v for k,v in enumerate(Tags)}

In [6]:
from tqdm import tqdm
def tagging(pos,start,end,type_):
    if end-start==1:
        return tag2id['S_'+type_]
    elif pos==start:
        return tag2id['B_'+type_]
    elif pos==end-1:
        return tag2id['E_'+type_]
    return tag2id['M_'+type_]
vocab=Counter()
texts=[]
labels=[]
length=[]
n=0
for i in tqdm(range(len(papers))):
    line=papers[i]
    text=line['text'].lower()
    if len(text)>150:
        continue
    for word in text:
        vocab[word]+=1
    for spo in line['spo_list']:
        object_type=spo['object_type']
        subject_type=spo['subject_type']
        object_=spo['object'].lower()
        subject_=spo['subject'].lower()
        label=[0]*len(text)
        
        if object_type not in entity_type and subject_type not in entity_type:
            continue
        if object_type in entity_type:
            try:
                object_start=text.index(object_)
                object_end=object_start+len(object_)
                for pos in range(object_start,object_end):
                    label[pos]=tagging(pos,object_start,object_end,object_type)
            except:
                continue

        if subject_type in entity_type:
            try:
                subject_start=text.index(subject_)
                subject_end=subject_start+len(subject_)
                for pos in range(subject_start,subject_end):
                    label[pos]=tagging(pos,subject_start,subject_end,subject_type)
            except:
                continue
        length.append(len(text))

        texts.append(text)
        labels.append(label)

            


100%|███████████████████████████████████████████████████████████████████████| 173108/173108 [00:10<00:00, 16521.38it/s]


In [8]:
vocab_list=['<PAD>','<UNK>']
for word,num in vocab.most_common():
    if num>=3:
        vocab_list.append(word)
        
word2id={k:v for v,k in enumerate(vocab_list)}
X=np.zeros((len(texts),150))
for i in tqdm(range(len(texts))):
    for j in range(len(texts[i])):
         X[i][j]=word2id.get(texts[i][j],word2id['<UNK>'])
y=np.zeros((len(texts),150))
for i in range(len(labels)):
    for j in range(len(labels[i])):
        y[i][j]=labels[i][j]

100%|███████████████████████████████████████████████████████████████████████| 340629/340629 [00:18<00:00, 17971.18it/s]


In [11]:
np.save('C://some tensorflow trick//relation//entity//X.npy',X)
np.save('C://some tensorflow trick//relation//entity//y.npy',y)
np.save('C://some tensorflow trick//relation//entity///texts.npy',texts)

np.save('C://some tensorflow trick//relation//entity//length.npy',length)
np.save('C://some tensorflow trick//relation//entity///Tags.npy',Tags)
np.save('C://some tensorflow trick//relation//entity//vocab.npy',vocab_list)

In [22]:
from collections import Counter
counter=Counter()
for paper in papers:
    for spo in paper['spo_list']:
        counter[spo['predicate']]+=1
        

In [36]:
save_relation_type=[]
for t,n in counter.most_common(15):
    save_relation_type.append(t)#不要出版社,民族